### Подключение библиотек 

In [1]:
import os
import re
import io
import json
from tika import parser

/usr/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.2) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


### Преобразование json-файлов в txt-для публикации

In [4]:
input_path = 'output_files/'
output_path = 'output_files_txt/'
files = os.listdir(input_path)
for file in files:
    json_file = open(input_path + '/' + file, "r")
    loaded_model_json = json_file.read()
    json_file.close()
    text = json.JSONDecoder().decode(loaded_model_json)
    f_name, f_extension = os.path.splitext(file)
    f = open(output_path + f_name + '.txt', 'w')
    f.write(str(text))
    f.close()

### Конвертация из pdf в txt

In [ ]:
path = 'Статьи/'
all_dirs = os.listdir(path)
for one_dir in all_dirs:
    new_path = 'Статьи - txt/' + one_dir + '/'
    print(new_path)
    if os.path.exists(new_path) == False:
        os.mkdir(new_path)    
    all_files = os.listdir(path + one_dir + '/')
    for file in all_files:
        raw = parser.from_file(path + one_dir + '/' + file)        
        f_name, f_extension = os.path.splitext(file)        
        f = open(new_path + f_name + '.txt', 'w')
        f.write(raw['content'])
        f.close()

### Приведение текстов к тому, чтобы их было удобно читать 

- убираю пробелы в начале новой строки
- убираю лишние пустые строки 
- склеиваю те переносы, которые через одну пустую строку
- склеиваю оставшиеся переносы
- убираю символы табуляции
- убираю лишние пробелы
- склеиваю разорванные строки через пустую строку или без неё
- удаляю оставшиеся отдельные строки, в которых несколько символов (скорее всего нетекстовая информация)

In [42]:
def remove_excess_enters(text):    
    text = re.sub('\\n +', '\\n', text)
    text = re.sub('\\n(\\n)+', '\\n\\n', text)
    text = re.sub('\-\\n\\n', '', text)
    text = re.sub('\-\\n', '', text)
    text = re.sub('\t', ' ', text)
    text = re.sub(' +', ' ', text)    
    s1 = re.findall('. ?\\n.', text)
    for i in s1:
        s2 = re.split(r'\n', i)
        s3 = str(str(s2[0]) + ' ' + str(s2[1]))
        try:
            text = re.sub(i, s3, text)
        except:
            print('WRONG-1')
        
    s1 = re.findall('[\w,;:"()«»-]{15,} ?\\n\\n[\w"()«»]', text)
    for i in s1:
        s2 = re.split(r'\n\n', i)
        s3 = str(str(s2[0]) + ' ' + str(s2[1]))
        try:
            text = re.sub(i, s3, text)
        except:
            print('WRONG-2')            
    text = re.sub('\\n{2}.{,10}\\n{2}', '\\n\\n', text)
    return text

### Предобработка файлов

In [ ]:
path = 'Статьи по языкам/Русский/TXT/'
new_path = 'Статьи по языкам/Русский/TXT-предобработка/'

if os.path.exists(new_path) == False:
    os.mkdir(new_path)  

all_dirs = os.listdir(path)
for one_dir in all_dirs:
    new_path_for_dir = new_path + one_dir + '/'    
    if os.path.exists(new_path_for_dir) == False:
        os.mkdir(new_path_for_dir)  
        
    all_files = os.listdir(path + one_dir + '/')
    for file in all_files:        
        f = open(path + one_dir + '/' + file, 'r')
        text = f.read()
        f.close()        
        text = remove_excess_enters(text)        
        f = open(new_path_for_dir + file, 'w')
        f.write(text)
        f.close()